In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os
import re

from functools import reduce

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
routine_clean = pd.read_csv('../../data/02_intermediate/routine_clean.csv')
astm = pd.read_csv('../../data/01_raw/ASTM_fuel.csv')
astm.columns = ['Date', 'TN_retailers_seasons', 'TN_distributor_seasons',
       'vapor_liquid_minC_retail', 'distillation_50_minC _retail',
       'distillation_50_maxC_retail', 'vapor_pressure_maxC_retail',
       'vapor_liquid_minC_dist', 'distillation_50_minC_dist',
       'distillation_50_maxC_dist', 'vapor_pressure_maxC_dist']

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Let's write a function! 

In [6]:
def clean_dataset_to_intermediate(dataset):
    gasoline = routine_clean.loc[routine_clean['Prod']=='Gasoline']

    gasoline_compliance = gasoline.loc[(gasoline['Compliance']=='Y')
                                       |(gasoline['Compliance']=='N')]
    gasoline_compliance['DateSampled'] = pd.to_datetime(gasoline_compliance['DateSampled'])
    # Let's reduce the number of tests to the three that we are interested in testing
    gasoline_compliance = gasoline_compliance.loc[(gasoline_compliance['Test']=='Distillation 50%') 
                            | (gasoline_compliance['Test']=='Vapor Pressure') 
                            | (gasoline_compliance['Test']=='Vapor-Liquid Ratio')]
    # Let's keep the first duplicates
    gasoline_compliance.drop_duplicates(inplace=True)
    gasoline_compliance.reset_index(drop=True, inplace=True)
    # create multilevel index
    gasoline_compliance.set_index(['Sample', 'Test'], inplace=True)
    # unstack on the inner undex (test)
    gasoline_compliance = gasoline_compliance.unstack(level=1)

    prod = gasoline_compliance['Prod']
    # let's create a dataframe for each product 
    prod.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    prod.reset_index(inplace=True)
    prod.rename(columns={'Vapor-Liquid Ratio':'prod'}, inplace=True)

    datesampled = gasoline_compliance['DateSampled']
    datesampled.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    datesampled.reset_index(inplace=True)
    datesampled.rename(columns={'Vapor-Liquid Ratio':'datesampled'}, inplace=True)

    grade = gasoline_compliance['Grade']

    grade.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    grade.reset_index(inplace=True)
    grade.rename(columns={'Vapor-Liquid Ratio':'grade'}, inplace=True)

    supplier = gasoline_compliance['Supplier']

    supplier.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    supplier.reset_index(inplace=True)
    supplier.rename(columns={'Vapor-Liquid Ratio':'supplier'}, inplace=True)

    facilityname = gasoline_compliance['FacilityName']

    facilityname.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    facilityname.reset_index(inplace=True)
    facilityname.rename(columns={'Vapor-Liquid Ratio':'facilityname'}, inplace=True)

    siteaddress = gasoline_compliance['SiteAddress']

    siteaddress.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
    siteaddress.reset_index(inplace=True)
    siteaddress.rename(columns={'Vapor-Liquid Ratio':'siteaddress'}, inplace=True)

    units = gasoline_compliance['Units']
    units.reset_index(inplace=True)
    units.rename(
        columns={'Distillation 50%':'units_dist_50', 
                 'Vapor Pressure':'units_vap_pressure', 
                 'Vapor-Liquid Ratio':'units_vap_liq_pressure'}, inplace=True)

    method = gasoline_compliance['Method']

    method.reset_index(inplace=True)
    method.rename(
        columns={'Distillation 50%':'method_dist_50', 
                 'Vapor Pressure':'method_vap_pressure', 
                 'Vapor-Liquid Ratio':'method_vap_liq_pressure'}, inplace=True)

    result = gasoline_compliance['Result']

    result.reset_index(inplace=True)
    result.rename(
        columns={'Distillation 50%':'result_dist_50', 
                 'Vapor Pressure':'result_vap_pressure', 
                 'Vapor-Liquid Ratio':'result_vap_liq_pressure'}, inplace=True)

    minresults = gasoline_compliance['MinResult']

    minresults.reset_index(inplace=True)
    minresults.rename(
        columns={'Distillation 50%':'minresults_dist_50', 
                 'Vapor Pressure':'minresults_vap_pressure', 
                 'Vapor-Liquid Ratio':'minresults_vap_liq_pressure'}, inplace=True)

    maxresults = gasoline_compliance['MaxResult']

    maxresults.reset_index(inplace=True)
    maxresults.rename(
        columns={'Distillation 50%':'maxresults_dist_50', 
                 'Vapor Pressure':'maxresults_vap_pressure', 
                 'Vapor-Liquid Ratio':'maxresults_vap_liq_pressure'}, inplace=True)

    compliance = gasoline_compliance['Compliance']

    compliance.reset_index(inplace=True)
    compliance.rename(
        columns={'Distillation 50%':'compliance_dist_50', 
                 'Vapor Pressure':'compliance_vap_pressure', 
                 'Vapor-Liquid Ratio':'compliance_vap_liq_pressure'}, inplace=True)

    df = [prod, datesampled, grade, supplier, facilityname, 
                   siteaddress, units, method, result, minresults, maxresults, compliance]

    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Sample'],
                                                how='outer'), df)
    
    return df_merged


In [5]:
len(df_merged)

20658

In [ ]:
gasoline = routine_clean.loc[routine_clean['Prod']=='Gasoline']

In [ ]:
gasoline_compliance = gasoline.loc[(gasoline['Compliance']=='Y')
                                   |(gasoline['Compliance']=='N')]

In [ ]:
print('Number of unique Samples in the gasoline_compliance Dataset: ', len(gasoline_compliance.Sample.unique()))

**Gasoline Tests w. Y/N Compliance** (only focusing on the bold tests moving forward) 
1.	Motor Octane Number
1.	Distillation Residue
1.	Distillation 90%
1.	Distillation E.P.
1.	Driveability Index
1.	Workmanship
1.	Distillation 10%
1.	**<font color='red'>Distillation 50%</font>**
1.	**<font color='red'>Vapor Pressure</font>**
1.	**<font color='red'>Vapor-Liquid Ratio</font>**
1.	Antiknock Index
1.	Phase Separation
1.	Ethanol
1.	Total Oxygen
1.	Distillation 60%
1.	Distillation 5%
1.	Distillation Loss
1.	Research Octane Number
1.	Distillation 80%
1.	Relative Density
1.	Distillation 95%
1.	Distillation 20%
1.	Methanol
1.	Distillation 40%
1.	Distillation Recovery
1.	API Gravity

In [ ]:
gasoline_compliance['DateSampled'] = pd.to_datetime(gasoline_compliance['DateSampled'])

In [ ]:
gasoline_compliance.Test.unique()

Let's reduce the number of tests to the three that we are interested in testing (Distillation 50%, Vapor Pressure, and Vapor-Liquid Ratio)

In [ ]:
gasoline_compliance = gasoline_compliance.loc[(gasoline_compliance['Test']=='Distillation 50%') 
                        | (gasoline_compliance['Test']=='Vapor Pressure') 
                        | (gasoline_compliance['Test']=='Vapor-Liquid Ratio')]

In [ ]:
gasoline_compliance.head(3)

Number of unique samples in the dataset

In [ ]:
len(gasoline_compliance.Sample.unique())

### There are 144 duplicated rows in this dataset. Let's keep the first occurance. 

In [ ]:
gasoline_compliance.duplicated(subset=None, keep='first').sum()

In [ ]:
gasoline_compliance.drop_duplicates(inplace=True)

In [ ]:
gasoline_compliance.reset_index(drop=True, inplace=True)

### Let's unstack this dataset

At the moment, every sample takes up three rows (each test has it's own dataset). Let's make sure that each sample has it's own row. 

In [ ]:
# create multilevel index
gasoline_compliance.set_index(['Sample', 'Test'], inplace=True)

In [ ]:
# unstack on the inner undex (test)
gasoline_compliance = gasoline_compliance.unstack(level=1)

now let's save each sub-dataframe into it's own dataframe so that we can re-name the columns

In [ ]:
prod = gasoline_compliance['Prod']

In [ ]:
prod.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
prod.reset_index(inplace=True)
prod.rename(columns={'Vapor-Liquid Ratio':'prod'}, inplace=True)

In [ ]:
datesampled = gasoline_compliance['DateSampled']
datesampled.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
datesampled.reset_index(inplace=True)
datesampled.rename(columns={'Vapor-Liquid Ratio':'datesampled'}, inplace=True)

In [ ]:
grade = gasoline_compliance['Grade']

In [ ]:
grade.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
grade.reset_index(inplace=True)
grade.rename(columns={'Vapor-Liquid Ratio':'grade'}, inplace=True)

In [ ]:
supplier = gasoline_compliance['Supplier']

In [ ]:
supplier.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
supplier.reset_index(inplace=True)
supplier.rename(columns={'Vapor-Liquid Ratio':'supplier'}, inplace=True)

In [ ]:
facilityname = gasoline_compliance['FacilityName']

In [ ]:
facilityname.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
facilityname.reset_index(inplace=True)
facilityname.rename(columns={'Vapor-Liquid Ratio':'facilityname'}, inplace=True)

In [ ]:
siteaddress = gasoline_compliance['SiteAddress']

In [ ]:
siteaddress.drop(['Distillation 50%', 'Vapor Pressure'], inplace=True, axis=1)
siteaddress.reset_index(inplace=True)
siteaddress.rename(columns={'Vapor-Liquid Ratio':'siteaddress'}, inplace=True)

In [ ]:
units = gasoline_compliance['Units']
units.reset_index(inplace=True)
units.rename(
    columns={'Distillation 50%':'units_dist_50', 
             'Vapor Pressure':'units_vap_pressure', 
             'Vapor-Liquid Ratio':'units_vap_liq_pressure'}, inplace=True)


In [ ]:
method = gasoline_compliance['Method']

In [ ]:
method.reset_index(inplace=True)
method.rename(
    columns={'Distillation 50%':'method_dist_50', 
             'Vapor Pressure':'method_vap_pressure', 
             'Vapor-Liquid Ratio':'method_vap_liq_pressure'}, inplace=True)

In [ ]:
result = gasoline_compliance['Result']

In [ ]:
result.reset_index(inplace=True)
result.rename(
    columns={'Distillation 50%':'result_dist_50', 
             'Vapor Pressure':'result_vap_pressure', 
             'Vapor-Liquid Ratio':'result_vap_liq_pressure'}, inplace=True)

In [ ]:
minresults = gasoline_compliance['MinResult']

In [ ]:
minresults.reset_index(inplace=True)
minresults.rename(
    columns={'Distillation 50%':'minresults_dist_50', 
             'Vapor Pressure':'minresults_vap_pressure', 
             'Vapor-Liquid Ratio':'minresults_vap_liq_pressure'}, inplace=True)

In [ ]:
maxresults = gasoline_compliance['MaxResult']

In [ ]:
maxresults.reset_index(inplace=True)
maxresults.rename(
    columns={'Distillation 50%':'maxresults_dist_50', 
             'Vapor Pressure':'maxresults_vap_pressure', 
             'Vapor-Liquid Ratio':'maxresults_vap_liq_pressure'}, inplace=True)

In [ ]:
compliance = gasoline_compliance['Compliance']

In [ ]:
compliance.reset_index(inplace=True)
compliance.rename(
    columns={'Distillation 50%':'compliance_dist_50', 
             'Vapor Pressure':'compliance_vap_pressure', 
             'Vapor-Liquid Ratio':'compliance_vap_liq_pressure'}, inplace=True)

In [ ]:
df = [prod, datesampled, grade, supplier, facilityname, 
               siteaddress, units, method, result, minresults, maxresults, compliance]

In [ ]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Sample'],
                                            how='outer'), df)

### Let's try to make a dummy variable for location

In [ ]:
print('Number of unique addresses in the dataset: ', len(df_merged['siteaddress'].unique()))

In [ ]:
print('Percent of samples with missing address: ', df_merged.siteaddress.isna().sum()/len(df_merged)*100)

#### let's make a zipcode column and then geocode the siteaddress column

In [ ]:
# df_merged['str_split'] = df_merged.siteaddress.str.split('Tn')

In [ ]:
# df_merged['zipcode'] = df_merged.str_split.str.get(1)

In [ ]:
# print('Percentage of zipcode column that is empty: ', df_merged.zipcode.isna().sum()/len(df_merged)*100)

In [ ]:
# df_merged.drop(columns=['str_split'], inplace=True)
# df_merged['city'] = ', Tn'
# df_merged['siteaddress_city'] = df_merged['siteaddress'] + df_merged['city']
# df_merged.drop(columns=['city'], inplace=True)

### Let's check out the target variable

In [ ]:
print('Compliance Outcomes dist 50: ', '\n', df_merged.compliance_dist_50.value_counts())

In [ ]:
print('dist 50 nan count: ', df_merged.compliance_dist_50.isna().sum())

In [ ]:
print('Compliance outcome vap liq press: ', '\n', df_merged.compliance_vap_liq_pressure.value_counts())

In [ ]:
print('Vap liq pressure nan count: ', df_merged.compliance_vap_liq_pressure.isna().sum())

In [ ]:
print('Compliance outcome vap press: ','\n', df_merged.compliance_vap_pressure.value_counts())

In [ ]:
print('Vap pressure nan count: ',df_merged.compliance_vap_pressure.isna().sum())

From our expert interview, we have determined that the nan results in the compliance rows are test results that were inconclusive. Below I have two blocks of code. 

Block 1: this converts the nan values to None (later if we want to better understand this catagory then we can use block 1 to encude the variable). 

Block 2: this drops the nan values for the target variable. This allows for analysis in the model building phase. 

Block 1: 
```python
df_merged['compliance_dist_50'] = df_merged['compliance_dist_50'].replace(np.nan, 'None')
df_merged['compliance_vap_liq_pressure'] = df_merged['compliance_vap_liq_pressure'].replace(np.nan, 'None')
df_merged['compliance_vap_pressure'] = df_merged['compliance_vap_pressure'].replace(np.nan, 'None')
```

In [ ]:
# Block 2: 
# df_merged.dropna(subset=['compliance_dist_50'], inplace=True)
# df_merged.dropna(subset=['compliance_vap_pressure'], inplace=True)
# df_merged.dropna(subset=['compliance_vap_liq_pressure'], inplace=True)

How many grades of gasoline does each supplier have

In [ ]:
df_merged.groupby(['supplier', 'grade']).count()

In [ ]:
supplier_grade = df_merged.groupby('supplier').grade.nunique().to_frame()
supplier_grade.reset_index(inplace=True)
print('Half of Suppliers only supply one grade of Gasoline: ', len(supplier_grade.loc[supplier_grade['grade']>1])/len(supplier_grade))

In [ ]:
len(supplier_grade.loc[supplier_grade['grade']>1])/len(supplier_grade)

In [ ]:
df_merged.to_csv('../../data/03_processed/gasoline_processed.csv', index=False)